In [2]:
from pyspark.sql import SparkSession 

# GroupBy and Aggregate Functions

In [3]:
spark = SparkSession.builder.appName('aggs').getOrCreate()

In [4]:
df = spark.read.csv('./data/sales_info.csv', inferSchema=True, header=True)

In [5]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [7]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [8]:
df.groupBy("Company")

In [10]:
df.groupBy("Company").count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [12]:
df.agg({'Sales':'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [13]:
group_data = df.groupBy("Company")

In [14]:
group_data.agg({'Sales':'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [15]:
from pyspark.sql.functions import countDistinct,avg,stddev

In [19]:
df.select(countDistinct('Sales').alias('Average Sales')).show()   # You import and pass that in to a df.select

+-------------+
|Average Sales|
+-------------+
|           11|
+-------------+



In [20]:
df.select(stddev('Sales')).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [21]:
from pyspark.sql.functions import format_number

In [22]:
std = df.select(stddev("Sales").alias('std'))

In [24]:
std.select(format_number('std',2)).show()

+---------------------+
|format_number(std, 2)|
+---------------------+
|               250.09|
+---------------------+



In [25]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [27]:
df.orderBy("Sales").show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [28]:
df.orderBy(df["Sales"].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



# Missing Data

In [29]:
from pyspark.sql import SparkSession

In [30]:
spark = SparkSession.builder.appName("miss").getOrCreate()

In [32]:
df = spark.read.csv("./data/ContainsNull.csv",inferSchema=True,header=True)

In [33]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [35]:
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [36]:
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [37]:
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [38]:
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [41]:
df.na.fill('Fill').show()   # Spark can infer the data type

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| Fill| null|
|emp3| Fill|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [40]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [42]:
df.na.fill("Fill",subset=["Name"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| Fill| null|
|emp3| Fill|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [43]:
from pyspark.sql.functions import mean

In [44]:
mean_val = df.select(mean(df["Sales"])).collect()

In [48]:
mean_sales = mean_val[0][0]

In [49]:
df.na.fill(mean_sales,subset=["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [50]:
df.na.fill(df.select(mean(df["Sales"])).collect()[0][0],["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



# Dates and Timestamps

In [51]:
from pyspark.sql import SparkSession

In [52]:
spark = SparkSession.builder.appName("dates").getOrCreate()

In [53]:
df = spark.read.csv("./data/appl_stock.csv", inferSchema=True, header=True)

In [54]:
df.head(1)

[Row(Date=datetime.date(2010, 1, 4), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)]

In [55]:
from pyspark.sql.functions import dayofmonth, hour, dayofyear, month, year, weekofyear,format_number,date_format

In [58]:
df.select(dayofmonth(df['Date']).alias("Date")).show()

+----+
|Date|
+----+
|   4|
|   5|
|   6|
|   7|
|   8|
|  11|
|  12|
|  13|
|  14|
|  15|
|  19|
|  20|
|  21|
|  22|
|  25|
|  26|
|  27|
|  28|
|  29|
|   1|
+----+
only showing top 20 rows



In [62]:
#df.select(year(df['Date'])).show()
newdf = df.withColumn("Year", year(df['Date']))

In [68]:
result = newdf.groupby("Year").mean().select("Year","avg(Close)")

In [71]:
new = result.withColumnRenamed("avg(Close)", "Average Closing Price")

In [77]:
new.select(['Year', format_number("Average Closing Price",2).alias("Avg Close")]).show()

+----+---------+
|Year|Avg Close|
+----+---------+
|2015|   120.04|
|2013|   472.63|
|2014|   295.40|
|2012|   576.05|
|2016|   104.60|
|2010|   259.84|
|2011|   364.00|
+----+---------+

